# Chapter 4

<a target="_blank" href="https://colab.research.google.com/github/CausalAILab/causalai-book/blob/main/chapter4.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Simpson's Paradox

Consider a medical setting with binary variables where:
- $X = 1$ indicates the patient took a certain drug for a year
- $Y = 1$ indicates that the patient recovered after one year of taking the drug
- $Z = 1$ indicates the patient is female

In [2]:
import os
import sys

if 'google.colab' in sys.modules:
    os.system("wget -O scm.py https://raw.githubusercontent.com/CausalAILab/causality-book/main/scm.py")

print(sys.version)

3.11.8 | packaged by conda-forge | (main, Feb 16 2024, 20:49:36) [Clang 16.0.6 ]


In [3]:
from sympy import symbols, Function, Symbol

from scm import SymbolicSCM

## Definition 4.1.1. Interventional Distribution

Given two disjoint sets of variables, $\mathbf X$ and $\mathbf Y$, the causal effect of $\mathbf X$ on $\mathbf Y$, denoted, $P(\mathbf y | do(\mathbf X = \mathbf x))$, is a function from $\mathbf X$ to the space of probability distributions of $\mathbf Y$. For each realization, $\mathbf X = \mathbf x$, $P(\mathbf y | do(\mathbf x))$ gives the probability of $\mathbf Y = \mathbf y$ induced by deleting from the model all equations corresponding to variables in $\mathbf X = \mathbf x$ and substituting $\mathbf X = \mathbf x$ in the remaining equations.

### Example 4.3 - Sprinkler

Consider a classic setting where a robot is in charge of maintaining
the backyard healthy and nourished throughout the entire year. The robot keeps track of the season
($Se$), sprinkler ($Sp$), rain ($Ra$), the wetness of the floor ($We$), and whether it is slippery ($Sl$). The
causal diagram $G$ is shown below and represents the following collection of mechanisms:

$$
\begin{align}
F &= \begin{cases}
    Se &\gets f_{Se}(U_{Se}) \\ 
    Sp &\gets f_{Sp}(Se, U_{Sp}) \\ 
    Ra &\gets f_{Ra}(Se, U_{Ra}) \\ 
    We &\gets f_{We}(Sp, Ra, U_{We}) \\ 
    Sl &\gets f_{Sl}(We, U_{Sl}) \\ 
\end{cases} \\
\end{align}
$$

for some $P(\mathbf U)$.

![Figure 4.6](Fig-4-6.png)

In [4]:
v = se, sp, ra, we, sl = symbols('Se Sp Ra We Sl')
f = f_se, f_sp, f_ra, f_we, f_sl = [Function(f'f_{{{k}}}') for k in v]
u = u_se, u_sp, u_ra, u_we, u_sl = [Symbol(f'u_{{{k}}}') for k in v]

m = SymbolicSCM(
    f={
        se: f_se(u_se),
        sp: f_sp(se, u_sp),
        ra: f_ra(se, u_ra),
        we: f_we(sp, ra, u_we),
        sl: f_sl(we, u_sl),
    },
    pu = {
        u_se: 0.5,
        u_sp: 0.5,
        u_ra: 0.5,
        u_we: 0.5,
        u_sl: 0.5,
    }
)